In [1]:
start = 5025
base_format = "https://www.ptt.cc/bbs/Stock/index{index}.html"
page = 2500

In [2]:
from tqdm import tqdm, trange

In [3]:
import requests
from bs4 import BeautifulSoup

targets = []
for p in tqdm(range(page)):
    response = requests.get(base_format.format(index=str(start - p)),
                            cookies={"over18":"1"})
    html = BeautifulSoup(response.text, "lxml")
    entries = html.find_all("div", class_="r-ent")
    for e in entries:
        t = e.find("div", class_="title").find("a")
        if t is not None:
            title = t.text
            if title.startswith("[") and not title.startswith("[公告]"):
                targets.append((title, t["href"]))
                
print("總共:", len(targets), "篇文章")
print("前十篇:", targets[:10])

100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [04:10<00:00,  9.99it/s]

總共: 35037 篇文章
前十篇: [('[新聞] 油價又高潮！OPEC驚人絕招 死空頭小心', '/bbs/Stock/M.1588305033.A.526.html'), ('[請益] 我有1000000元，該如何操作？', '/bbs/Stock/M.1588305201.A.E5E.html'), ('[請益] 原油期權6/7月停開新倉', '/bbs/Stock/M.1588305794.A.6F5.html'), ('[新聞] 慘！疫情空襲 勞動基金前3月大賠4,712億', '/bbs/Stock/M.1588306449.A.1D4.html'), ('[其他] 一個月前畢業的現在在幹嘛?', '/bbs/Stock/M.1588306885.A.D49.html'), ('[請益] 聯發科為什麼一直漲?', '/bbs/Stock/M.1588308183.A.1AB.html'), ('[新聞] 亞馬遜預估疫情恐耗盡Q2獲利 股價聞訊重', '/bbs/Stock/M.1588311131.A.DAA.html'), ('[新聞]東貝與中華郵政完成三重「台北科技城」買', '/bbs/Stock/M.1588311261.A.81B.html'), ('[請益] 紓困一百萬使用', '/bbs/Stock/M.1588311299.A.62A.html'), ('[請益] 這波反彈2千點 大家獲益多少?', '/bbs/Stock/M.1588313050.A.FFF.html')]


In [4]:
def save_article(article, dn):
    article_base = "https://www.ptt.cc"
    article_url = article_base + article
    response = requests.get(article_url, cookies={"over18":"1"})
    html = BeautifulSoup(response.text, "lxml")

    main = html.find("div", id="main-content")
    metas = main.find_all("span", class_="article-meta-value")
    user_id = metas[0].text.split(" ")[0]
    title = metas[2].text
    pattern = re.compile(r"\[.+\]")
    article_type = (re.match(pattern, title)
               .group()
               .replace("[", "")
               .replace("]", ""))
    article_title = re.split(pattern, title)[-1].lstrip(" ")
    article_time = str(datetime.strptime(metas[3].text, 
                                         "%a %b %d %H:%M:%S %Y"))

    pushes = main.find_all("div", class_="push")
    article_push = []
    article_score = 0

    for m in main.find_all("div", class_="article-metaline"):
        m.extract()
    for m in main.find_all("div", class_="article-metaline-right"):
        m.extract()    

    for p in pushes:
        tag = p.find("span", class_="push-tag")
        if "推" in tag.text:
            push_score = 1
            article_score = article_score + 1
        elif "噓" in tag.text:
            push_score = -1
            article_score = article_score - 1
        else:
            push_score = 0
        push_id = p.find("span", class_="push-userid").text
        push_content = p.find("span", class_="push-content").text
        article_push.append({"type":push_score,
                             "id":push_id,
                             "content":push_content})
        p.extract()

    ip = None
    for f in main.find_all("span", class_="f2"):
        if "發信站:" in f.text:
            pat = re.compile("[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+")
            ip = re.search(pat, f.text).group()
            f.extract()
        elif "文章網址:" in f.text:
            f.extract()

    data = {
        "ptt_id":user_id,
        "post_url":article_url,
        "post_type":article_type,
        "post_title":article_title,
        "post_time":article_time,
        "post_score":article_score,
        "post_pushes":article_push,
        "post_ip":ip,
        "post_content":main.text
    }

    fn = article_url.split("/")[-1].split(".html")[0]
    with open(os.path.join(dn, fn + ".json"), "w", encoding="utf-8") as f:
        json.dump(data, f)

In [5]:
import re
from datetime import datetime
import json
import os

dn = "pttdata"
if not os.path.exists(dn):
    os.makedirs(dn)

with tqdm(total=len(targets)) as pbar:
  for t in targets:
      try:
          save_article(t[1], dn)
      except Exception as e:
          print(e)
      pbar.update(1)

  0%|▏                                                                            | 88/35037 [00:10<1:35:07,  6.12it/s]

'NoneType' object has no attribute 'text'


  0%|▎                                                                           | 126/35037 [00:14<1:28:58,  6.54it/s]

'NoneType' object has no attribute 'text'


  1%|▍                                                                           | 191/35037 [00:22<2:04:05,  4.68it/s]

...

100%|█████████████████████████████████████████████████████████████████████████▊| 34947/35037 [1:06:31<00:15,  5.75it/s]

'NoneType' object has no attribute 'text'


100%|█████████████████████████████████████████████████████████████████████████▉| 35024/35037 [1:06:39<00:01,  7.13it/s]

'NoneType' object has no attribute 'text'


100%|██████████████████████████████████████████████████████████████████████████| 35037/35037 [1:06:40<00:00,  8.76it/s]


# 使用資料

In [ ]:
import json
import pandas as pd

In [ ]:
input_file = open ('pttdata/M.1588305201.A.E5E.json')
json_array = json.load(input_file)

In [ ]:
json_array